In [1]:
import pandas as pd
from transformers import ViltConfig, ViltProcessor
import torch
from PIL import Image
import os
from transformers import ViltForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from PIL import Image
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader
import json

In [2]:
final_data=pd.read_csv(os.path.join(os.getcwd(),'test-data.csv'))
epochs = 101
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
import json
from torch.nn.functional import one_hot
# read the encodings.json file
with open(os.path.join(os.getcwd(),'tokens.json'), 'r') as f:
    data = json.load(f)

# extract the unique_answers, answer_to_index, and predicted_answers lists
unique_answers = data['unique_answers']
answer_to_index = data['answer_to_index']
predicted_answers = data['predicted_answers']

print("recieved one_hot_vectors")

recieved one_hot_vectors


In [4]:
len(unique_answers)

1421

In [5]:
len(answer_to_index)

1421

In [20]:
import torch
from PIL import Image
class VQADataset(torch.utils.data.Dataset):
  def __init__(self, processor):
    final_data = pd.read_csv('test-data.csv')
    self.questions = final_data['question']
    self.image_id = final_data['image']
    self.processor = processor
    self.answers = final_data['ans-hyp']

  def __len__(self):
    return len(self.image_id)

  def __getitem__(self,idx):
    # get image + text
    questions = self.questions[idx]
    img_path = os.path.join(os.getcwd(),'osfstorage-archive','VQA_RADImage',self.image_id[idx])
    image = Image.open(img_path)
    answer = self.answers[idx]
    
    encoding = self.processor(image, questions, padding="max_length", truncation=True, return_tensors="pt")

    # remove batch dimension
    for k,v in encoding.items():
      encoding[k] = v.squeeze()

    answer_words = answer.split('-')
    scores = list()
    if len(answer_words) > 1:
      scores = [(1/len(answer_words)) for _ in range(len(answer_words))]
    else: 
      scores = [1.0]
    # based on: https://github.com/dandelin/ViLT/blob/762fd3975c180db6fc88f577cf39549983fa373a/vilt/modules/objectives.py#L301
    targets = torch.zeros(len(answer_to_index))
    print(len(answer_to_index))
    scores_final = torch.tensor(scores)
    for ans, score in zip(answer_words,scores_final):
      targets[answer_to_index[ans]] = score
    encoding["labels"] = targets
    return encoding


In [21]:
dataset = VQADataset(processor=processor)

In [22]:
dataset[0].keys()

1421


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])

In [27]:
processor.decode(dataset[0]['input_ids'])

1421


'[CLS] where is the mass located? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [28]:
labels = torch.nonzero(dataset[0]['labels']).squeeze().tolist()

1421


In [29]:
print(labels)

[474, 485]


In [30]:
[unique_answers[label] for label in labels]

['plexus', 'choroid']

## Define model


In [31]:
from transformers import ViltForQuestionAnswering

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm")

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.1.weight', 'classifier.0.bias', 'classifier.0.weight', 'classifier.3.bias', 'classifier.1.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.classifier[3] = torch.nn.Linear(in_features=1536 , out_features=1421, bias=True)
model.to(device)

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

Next, we create a corresponding PyTorch DataLoader, which allows us to iterate over the dataset in batches.

Due to the fact that the processor resizes images to not necessarily the same size, we leverage the `pad_and_create_pixel_mask` method of the processor to pad the pixel values of a batch and create a corresponding pixel mask, which is a tensor of shape (batch_size, height, width) indicating which pixels are real (1) and which are padding (0).

## Inference

Let's verify whether the model has actually learned something:

## Test Set Evaluation

In [33]:
checkpoint = torch.load('/Users/raunakpandey/Downloads/ViLT_final_check_point_epoch95.pt', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model = model.eval()
# # - or -
# model.train()

model.to(device)

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [38]:
example = dataset[0]
print(example.keys())

1421
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])


In [39]:
processor.decode(example['input_ids'])

'[CLS] where is the mass located? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

Note that we need to apply a sigmoid activation on the logits since the model is trained using binary cross-entropy loss (as it frames VQA as a multi-label classification task).

In [40]:
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
outputs = model(**example)

In [41]:
logits = outputs.logits
predicted_classes = torch.sigmoid(logits)

probs, classes = torch.topk(predicted_classes, 5)

for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
  print(prob, unique_answers[class_idx])

0.45213961601257324 ventricles
0.35632842779159546 lateral
0.004177028313279152 and
0.0010874273721128702 cerebrum
0.0009801362175494432 anterior


## Open Ended Questions

In [30]:
image = Image.open('/Users/raunakpandey/Documents/programming/projects/med-flamingo/ViLT/osfstorage-archive/VQA_RADImage/synpic16174.jpg')
text = "is this a male body?"

In [31]:
# prepare inputs
encoding = processor(image, text, return_tensors="pt")
encoding.to(device)

# forward pass
outputs = model(**encoding)
logits = outputs.logits

predicted_classes = torch.sigmoid(logits)

probs, classes = torch.topk(predicted_classes, 5)

for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
  print(prob, unique_answers[class_idx])


0.9992679953575134 yes
0.0017395588802173734 no
2.0657535060308874e-05 maybe
1.9025181245524436e-05 midline
9.883326129056513e-06 right
